In [4]:
import quimb.tensor as qtn
from vmc_torch.experiment.tn_model import *
from vmc_torch.fermion_utils import *
import torch
from vmc_torch.hamiltonian_torch import *

Lx=4
Ly=2
N_f = int(Lx*Ly-2)
t=1
U=8
n_fermions_per_spin = (N_f//2, N_f//2)
H = spinful_Fermi_Hubbard_square_lattice_torch(Lx, Ly, t, U, N_f, pbc=False, n_fermions_per_spin=n_fermions_per_spin)
# SU in quimb
D = 4
seed = 0
symmetry = 'Z2'
spinless = False
peps = generate_random_fpeps(Lx, Ly, D=D, seed=seed, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
try:
    parse_edges_to_site_info = sr.utils.parse_edges_to_site_info
except AttributeError:
    parse_edges_to_site_info = sr.parse_edges_to_site_info
site_info = parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.float32))
peps_model = fTNModel(peps, max_bond=-1)
peps_model.num_params

640

In [5]:
D8_peps = generate_random_fpeps(Lx, Ly, D=4, seed=seed, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
D8_peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.float32))
peps_nn_model = fTN_backflow_attn_Tensorwise_Model_v1(D8_peps, max_bond=-1, embedding_dim=16, attention_heads=4, nn_final_dim=8)
peps_nn_model.num_params

15824

In [10]:
dtype = torch.float64
model = NNBF_attention(
    nsite=Lx*Ly,
    hilbert=H.hilbert,
    param_dtype=dtype,
    hidden_dim=64,
    embed_dim=16,
    nn_eta=0.0,
    attention_heads=4,
    phys_dim=4,
    spinflip_symmetry=False,
)

model.num_params

15888

In [9]:
sd = SlaterDeterminant(
    H.hilbert,
    param_dtype=dtype,
)
sd.num_params

96